In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle
import numpy as np

rftrain_vect = []
rftrain_op = []

with open("rftrain_vect","rb") as file:
    rftrain_vect = np.array(pickle.load(file))
with open("rftrain_op","rb") as file:
    rftrain_op = np.array(pickle.load(file))

X_test = []
y_test = []

with open("X_test","rb") as file:
    X_test = np.array(pickle.load(file))
with open("y_test","rb") as file:
    y_test = np.array(pickle.load(file))
import sys
md = 20
nest = 500
clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=md),
    n_estimators=nest
)
print("fitting model")
clf.fit(rftrain_vect,rftrain_op)
print("predicting..")
predz = clf.predict_proba(X_test)

with open("ADA_PREDICTIONS_md_"+str(md)+"_nest_"+str(nest),"wb") as file:
	pickle.dump(predz,file)

def map5eval2(preds,actual):
    predicted = []
    for i in range(preds.shape[0]):
        predicted.append(preds[i].argsort()[-5:][::-1])
    predicted = np.array(predicted)
    metric = 0.
    for i in range(5):
        metric += np.sum(actual==predicted[:,i])/(i+1)
        print("recommendation",i+1,np.sum(actual==predicted[:,i]))
    metric /= actual.shape[0]
    return 'MAP@5', metric, False

print("map5:",map5eval2(predz,y_test)[1])

fitting model
predicting..
recommendation 1 11721
recommendation 2 2608
recommendation 3 1478
recommendation 4 1103
recommendation 5 841
map5: 0.46538722222222223


In [2]:
import pickle
import numpy as np

def map5eval(preds, actual):
    predicted = []
    for i in range(preds.shape[0]):
        predicted.append(preds[i].argsort()[-5:][::-1])
    predicted = np.array(predicted)
    metric = 0.
    for i in range(5):
        metric += np.sum(actual==predicted[:,i])/(i+1)
        print("recommendation",i+1,":",np.sum(actual==predicted[:,i]))
    metric /= actual.shape[0]
    return metric

def map5eval2(predicted, actual):
    metric = 0.
    for i in range(5):
        metric += np.sum(actual==predicted[:,i])/(i+1)
#         print("recommendation",i+1,":",np.sum(actual==predicted[:,i]))
    metric /= actual.shape[0]
    return metric

xgb_preds = []
with open("XGB_PREDICTIONS_2","rb") as file:
    # xgb_preds = pickle.load(file)
    xgb_preds = pickle.load(file)
rf_preds = []
with open("RF_PREDICTIONS_FINALLL","rb") as file:
    rf_preds = pickle.load(file)
ada_preds = []
with open("ADA_PREDICTIONS_md_20_nest_500","rb") as file:
	ada_preds = pickle.load(file)
y_test = []
with open("y_test","rb") as file:
    y_test = np.array(pickle.load(file))

# print("RandomForest:")
# print("rf map5:",map5eval(rf_preds,y_test))
# print("XGBoost:")
# print("xgb map5:",map5eval(xgb_preds,y_test))
# print("adaboost:")
# print("ada map5:",map5eval(ada_preds,y_test))
# w1 = 2.3 # rf weight - 4.2: 0.4542 --> 4.23
# w2 = 2 # xgb weight - 2: 0.455
# w3 = 2.5 # ada weight
# w1 = 2.5
# w2 = 1.5
# w3 = 4
import time
tm=0
wm=[]
weights = []
test_map5s = []
train_map5s = []
for w1 in range(1,11):
    for w2 in range(1,11):
        for w3 in range(1,15):
            if w1==w2 and w2==w3 and w1!=1:
                continue
            print("w1:",w1,"w2:",w2,"w3:",w3)
            finprobs = []
            strtime = time.time()
            '''
            for i in range(20000):
                p1 = rf_preds[i].argsort()[-5:][::-1]
                p2 = xgb_preds[i].argsort()[-5:][::-1]
                p3 = ada_preds[i].argsort()[-5:][::-1]
                p1_scores = sorted(rf_preds[i])[-5:][::-1]
                p2_scores = sorted(xgb_preds[i])[-5:][::-1]
                p3_scores = sorted(ada_preds[i])[-5:][::-1]
                probs = [0]*100
                for k in range(5):
                    if p1[k]==p2[k] and p1[k] == p3[k]:
                        probs[p1[k]] += (w1*p1_scores[k]) + (w2*p2_scores[k]) + (w3*p3_scores[k])
                    elif p1[k]==p2[k]:
                        probs[p1[k]] += (w1*p1_scores[k]) + (w2*p2_scores[k])
                        probs[p3[k]] += (w3*p3_scores[k])
                    elif p2[k]==p3[k]:
                        probs[p2[k]] += (w2*p2_scores[k]) + (w3*p3_scores[k])
                        probs[p1[k]] += (w1*p1_scores[k])
                    elif p1[k]==p3[k]:
                        probs[p1[k]] += (w1*p1_scores[k]) + (w3*p3_scores[k])
                        probs[p2[k]] += (w2*p2_scores[k])
                    else:
                        probs[p1[k]] += (w1*p1_scores[k])
                        probs[p2[k]] += (w2*p2_scores[k])
                        probs[p3[k]] += (w3*p3_scores[k])

                probs = np.array(probs).argsort()[-5:][::-1]
                finprobs.append(probs)
            trmap5 = map5eval2(np.array(finprobs),y_test[:20000])
            print("train ensemble map5:",trmap5)'''
            finprobs=[]
            for i in range(20000,30000):
                p1 = rf_preds[i].argsort()[-5:][::-1]
                p2 = xgb_preds[i].argsort()[-5:][::-1]
                p3 = ada_preds[i].argsort()[-5:][::-1]
                p1_scores = sorted(rf_preds[i])[-5:][::-1]
                p2_scores = sorted(xgb_preds[i])[-5:][::-1]
                p3_scores = sorted(ada_preds[i])[-5:][::-1]
                probs = [0]*100
                for k in range(5):
                    if p1[k]==p2[k] and p1[k] == p3[k]:
                        probs[p1[k]] += (w1*p1_scores[k]) + (w2*p2_scores[k]) + (w3*p3_scores[k])
                    elif p1[k]==p2[k]:
                        probs[p1[k]] += (w1*p1_scores[k]) + (w2*p2_scores[k])
                        probs[p3[k]] += (w3*p3_scores[k])
                    elif p2[k]==p3[k]:
                        probs[p2[k]] += (w2*p2_scores[k]) + (w3*p3_scores[k])
                        probs[p1[k]] += (w1*p1_scores[k])
                    elif p1[k]==p3[k]:
                        probs[p1[k]] += (w1*p1_scores[k]) + (w3*p3_scores[k])
                        probs[p2[k]] += (w2*p2_scores[k])
                    else:
                        probs[p1[k]] += (w1*p1_scores[k])
                        probs[p2[k]] += (w2*p2_scores[k])
                        probs[p3[k]] += (w3*p3_scores[k])

                probs = np.array(probs).argsort()[-5:][::-1]
                finprobs.append(probs)
            tmap5 = map5eval2(np.array(finprobs),y_test[20000:30000])
            if tmap5 > tm:
                tm = tmap5
                wm = [w1,w2,w3]
            weights.append([w1,w2,w3])
            test_map5s.append(tmap5)
            print("test ensemble map5:",map5eval2(np.array(finprobs),y_test[20000:30000]), " - ",time.time()-strtime)
            print("------")
print(wm,tm)


In [7]:
map5eval(xgb_preds,y_test)

recommendation 1 : 9420
recommendation 2 : 2857
recommendation 3 : 1908
recommendation 4 : 1366
recommendation 5 : 1066


0.4013066666666667

In [8]:
map5eval(rf_preds,y_test)

recommendation 1 : 10985
recommendation 2 : 2695
recommendation 3 : 1843
recommendation 4 : 1308
recommendation 5 : 967


0.4489077777777778

In [9]:
map5eval(ada_preds,y_test)

recommendation 1 : 11529
recommendation 2 : 2497
recommendation 3 : 1613
recommendation 4 : 1104
recommendation 5 : 884


0.4589322222222222